In [ ]:
%%bash
pip install numpy-quaternion

In [69]:
import numpy as np
import quaternion
import matplotlib.pyplot as plt

from scipy import optimize

from camera_calibration_parsers import readCalibration
import image_geometry

from tf import transformations

import time
from copy import deepcopy

import cv2

from IPython.display import clear_output

In [10]:
from client_mk1.fast_image import Image
from client_mk1.color import Color
from client_mk1.feat_2d import Circle2D
from client_mk1.searchers import do_linear_search, do_binary_search

In [11]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# Useful resources

[OpenCV tutorial](http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_gui/py_video_display/py_video_display.html#playing-video-from-file)

![](http://wiki.ros.org/image_pipeline/CameraInfo?action=AttachFile&do=get&target=CameraCoords.png)

## Load camera configuration info

In [54]:
_, calibration = readCalibration("../calibration/stabilized_pixel.yaml")
camera = image_geometry.PinholeCameraModel()
camera.fromCameraInfo(calibration)

# Define types for 3D features

In [55]:
class Sphere3D(object):
    """
    Sphere3D represents a sphere known to exist in 3D map-space.
    """
    def __init__(self, center, radius, colorRange):
        self.center = np.array(center)
        self.radius = radius
        self.color = colorRange

        # TODO: this should be a scalar with more reasonable behavior
        self.confidence = True
        
    def project(self, pos, orientation):
        """
        Returns the coordinates of the given point in the camera image
        """
        relative_center = self.center - pos
        relative_center = np.dot(np.transpose(orientation), relative_center)
        
        center2D = camera.project3dToPixel(relative_center)
        x, y, w = relative_center
        radius2D = camera.fx() * self.radius / w
        
        return Circle2D(center2D, radius2D, self.color)


In [56]:
s = greenSphere
s.project(startPos,np.identity(3))

Circle(center=[ 1434.94873375   804.8589498 ], radius=94.973709711)

# Define hardcoded map for testing tracking

In [85]:
green = Color((30, 60, 100), (60, 255, 255))
orange = Color((5, 150, 130), (30, 255, 255))

greenCircle = Circle2D((1920/2, 1080/2), 400, green)
orangeCircle = Circle2D((1920/2, 1080/2), 400, orange)

greenSmall = Circle2D((1100, 700), 130, green)
orangeSmall = Circle2D((600, 700), 130, orange)

greenSphere = Sphere3D((.5,.3,0), .106, green)
orangeSphere = Sphere3D((-.5,.3,0), .106, orange)

startPos = np.array([0, 0, -2])
startList = list(startPos)+[1,0,0,0]

# Find 3D poses

In [75]:
def getDistFunc(spheres, foundCircles):
    assert len(spheres) == len(foundCircles)
    def distFunc(merged):
        """
        Takes a list of 7 numbers. The first three represent translational 
        position, and the last four form a rotation quaternion.
        Returns a list of error values.
        """
        position = np.array(merged[:3])
        quat = np.quaternion(*merged[3:])
        matrix = quaternion.as_rotation_matrix(quat)
        
        errors = [c.distance(s.project(position, matrix).center)
         for s, c in zip(spheres, foundCircles)]
        
        return errors + [quat.norm() - 1]
        
        
    return distFunc

In [76]:
quat = np.quaternion(1,2,3,4)

In [86]:
getDistFunc([greenSphere, orangeSphere], [greenSmall, orangeSmall]
           )(startList)

[350.97870817741534, 121.32608426204028, 0.0]

# Capture and handle video

In [66]:
FILE = '../test_data/smooth_1.mp4'
cv2.namedWindow('raw_frame')

In [ ]:
cap = cv2.VideoCapture(FILE)

def handleFrame(frame, features, verbose=False):
    image = Image(frame)

    for f in features:
        f.refine(image, verbose=verbose)
    
    return image


def main():
    features = deepcopy([greenSmall, orangeSmall])
    i=0
    while cap.isOpened():
        verbose = i%50 == 0
        if verbose:
            clear_output()
        i += 1
        
        startTime = time.time()
        ret, frame = cap.read()
        readTime = time.time()

        if frame is None or len(frame) <= 0:
            print "End of video file reached"
            break

        # Do work here
        image = handleFrame(frame, features, verbose=verbose)
        
        finishTime = time.time()

        for f in features:
            f.draw(frame)
            
        image.draw(frame)
        scaled = cv2.resize(frame, dsize=None, fx=0.5, fy=0.5)
        cv2.imshow('raw_frame', scaled)
        
        drawnTime = time.time()

        cv2.waitKey(1)
        
        if verbose:
            processingTime = (finishTime-readTime)
            print "{0:.1f}ms reading and drawing image, {1:.1f}ms ({3}fps) processing, {2} pixels accessed\n".format(
                (readTime-startTime + drawnTime-finishTime)*1000, 
                processingTime*1000, 
                len(image.pixelsAccessed),
                int(1/processingTime))
        
main()

# Profile things

In [ ]:
cap = cv2.VideoCapture(FILE)

def bench():
    features = deepcopy([greenSmall])
    for _ in range(60):
        ret, frame = cap.read()
        handleFrame(frame, features)
    
%lprun -r -f doBinarySearch bench()